#**install and imports**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling1D , Conv1D , Dropout , Activation , Embedding , Input , Lambda , Conv2D , MaxPooling2D , Flatten
# from keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import keras
from keras.regularizers import l2
from keras.layers import BatchNormalization

#**Load Data**

In [ ]:
# loaded_arr = np.loadtxt('/content/full_data.txt')

In [ ]:
# n = 5
# load_original_arr = loaded_arr.reshape(
#     loaded_arr.shape[0], loaded_arr.shape[1] // n, n)

In [ ]:
# load_original_arr.shape

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/programming/full_data.csv')
# df['mid']
# df.loc[:,'rsi1':]

In [ ]:
df.loc[:,'rsi1':] = df.loc[:,'rsi1':]
df = df.iloc[1:]

In [ ]:
df.shape

#**split the data**

In [ ]:
# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(load_original_arr.reshape(-1,1))
# scaled = scaled.reshape(load_original_arr.shape)
# scaled = load_original_arr

In [ ]:
g = df.groupby('Action')
df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

In [ ]:
X = df.loc[:,'rsi1':]
# X = X[sorted_columns]
X = X.values
y = df.loc[:,'Action'].values

In [ ]:
X_train , X_validation , y_train , y_validation = train_test_split(X,y,test_size = 0.2 , shuffle=True)

In [ ]:
mm_scaler = MinMaxScaler(feature_range=(0, 1))
X_train = mm_scaler.fit_transform(X_train)
X_validation = mm_scaler.fit_transform(X_validation)

In [ ]:
y_train = [y for y in y_train]
y_train = np.array(y_train)

y_validation = [y for y in y_validation]
y_validation = np.array(y_validation)

In [ ]:
y_train = to_categorical(y_train, 3)
y_validation = to_categorical(y_validation, 3)

In [ ]:
X_validation.shape , y_validation.shape

#**Visualizing Data**

In [ ]:
from PIL import Image as im

In [ ]:
for i in range(30):
  d = np.reshape(X_train, (X_train.shape[0] , 15, 15))[i]
  plt.imshow(d , interpolation='nearest')
  plt.show()
  print(y[i])

#**train CNNs model**

In [ ]:
X_train = X_train.reshape(X_train.shape[0] , 15 , 15 , 1)
X_validation = X_validation.reshape(X_validation.shape[0] , 15 , 15 , 1)

In [ ]:
def define_model():

  model = Sequential()

# convolutional layer
  model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))

# convolutional layer
  model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=(2,2)))
  model.add(Dropout(0.25))

# flatten output of conv
  model.add(Flatten())

# hidden layer
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(250, activation='relu'))
  model.add(Dropout(0.3))
# output layer
  model.add(Dense(10, activation='softmax'))

# compiling the sequential model
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
  return model

In [ ]:
model = define_model()
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_validation, y_validation), verbose=1)

In [ ]:
scores = model.evaluate(X_validation, y_validation, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
prediction = np.argmax(model.predict(X_validation), axis=-1)
print(prediction.tolist())

In [ ]:
scaler.inverse_transform(prediction)

In [ ]:
model.save_weights("model.h5")

#**train multiple models**

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))